<a href="https://colab.research.google.com/github/shahnbej/Langchain/blob/main/06_04_Presentation_ReAct_Prompting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install langchain openai==1.10.0 langchain-openai langchainhub duckduckgo-search wikipedia

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 8.4 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 5.2 MB/s eta 0:00:0

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


# **ReAct Prompting Framework Overview**

The ReAct framework was [introduced in a 2022 paper, ReAct: Synergizing Reasoning and Acting in Language Models, by researchers at Google](https://blog.research.google/2022/11/react-synergizing-reasoning-and-acting.html).

### 🌐 **ReAct (Reasoning + Acting) Explained**

- Blends reasoning with action in language models.

- Generates reasoning traces alongside task-specific actions.

### 🔗 **Capabilities of ReAct**

1. 🔄 **Interleaved Reasoning and Acting**: Fuses reasoning with actions for fluid updates.

2. 🌍 **External Interactions**: Accesses external sources like Wikipedia for more info.

3. 🧑‍🎨 **Human-like Problem Solving**: Offers easy-to-follow, logical trajectories.

4. 🛠️ **Wide-Ranging Use Cases**: Ideal for varied tasks like QA, text games, web browsing.

<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2210.03629/assets/x1.png" alt="Image Description" style="width:100%">
  <figcaption>Comparison of four prompting methods, (a) Standard, (b) Chain-of-thought (CoT, Reason Only), (c) Act-only, and (d) (Reason+Act), solving a HotpotQA question. (2) Comparison of (a) Act-only and (b) prompting methods to solve an interactive AlfWorld task. In both domains, the initial prompts of in-context examples are ommitted, and only show task solving trajectories generated by the model (Act, Thought) and the environment (Obs).</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2210.03629">Image Source</a>
</figure>


### 🌟 **Why ReAct Stands Out**

1. **Balanced Emphasis on Reasoning & Acting**: Surpasses models limited to just reasoning or action.

2. **Engages with External Data**: Pulls in real-time info for better accuracy.

3. **Boosted Transparency**: Step-by-step approach clarifies decision-making.


# 🤖 **Overview of ReAct in LangChain**

- ReAct is an agent type in LangChain.

- It uses the ReAct framework to pick tools based on their descriptions.

- Useful for chatbots, document question answering, etc.

- Context-aware for better response generation.

## 🚀 **Steps to Implement an Agent**

1. ✍️ **Prompt Template Creation**: Guide the agent on tasks and output formatting.

2. 🧩 **Output Parser Definition**: Convert LLM output into actionable responses.

3. 🧠 **LLM Setup**: Choose the preferred language model.

4. 🛑 **Stop Sequence Definition**: Instruct the LLM when to stop output.

5. 🤹 **Agent Setup**: Merge LLM, prompt template, output parser, and stop sequence.

6. 🏃 **Agent Usage**: Operate the agent using an executor with user inputs.

In [3]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain import hub
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain_community.tools import DuckDuckGoSearchResults
from langchain import Wikipedia

In [4]:
search = DuckDuckGoSearchResults()

search_tool = Tool(
    name="Current Event Search",
    description = "Useful for when you need to search about current events.",
    func=search.run
)

wiki = Wikipedia()

wiki_tool = Tool(
    name="Noun Search",
    description = "Useful for when you need to search for information about nouns.",
    func=wiki.search,
)

tool_belt = [search_tool, wiki_tool]

In [5]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [7]:
prompt.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

In [8]:
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [9]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [ ]:
prompt.input_variables

['agent_scratchpad', 'input', 'tool_names', 'tools']

In [10]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

In [12]:
agent = create_react_agent(llm, tool_belt, prompt)

In [14]:
agent_executor = AgentExecutor(agent=agent,
                               tools=tool_belt,
                               verbose=True,
                               handle_parsing_errors=True)

In [15]:
agent_executor.invoke({"input":"who is playing in the Super Bowl in 2025? where will it be played?"})



> Entering new AgentExecutor chain...
I should use the Current Event Search tool to find information about the Super Bowl in 2025.
Action: Current Event Search
Action Input: Super Bowl 2025 teams and location[snippet: Where is 2025 Super Bowl? Date: Sunday, Feb. 9 | Time: 6:30 p.m. ET Location: Caesars Superdome (New Orleans) TV: Fox | Stream: fubo The Superdome is an important location for the people of the city, title: Where is Super Bowl 2025? Date, time, location, stadium, logo revealed ..., link: https://www.cbssports.com/nfl/news/where-is-super-bowl-2025-date-time-location-stadium-logo-revealed-and-more-for-super-bowl-lix/], [snippet: Super Bowl Locations and Dates. Super Bowl LIX: Feb. 9, 2025 - Caesars Superdome (New Orleans, LA), home of the New Orleans Saints; Super Bowl LX: Feb. 8, 2026 - Levi's Stadium (Santa Clara, CA ..., title: Super Bowl locations, dates for 2025, 2026, 2027 | FOX Sports, link: https://www.foxsports.com/stories/nfl/super-bowl-locations-dates], [snippe

{'input': 'who is playing in the Super Bowl in 2025? where will it be played?',
 'output': 'The Super Bowl in 2025 will be played at Caesars Superdome in New Orleans. The teams playing are not specified in the search results.'}

In [ ]:
agent_executor.invoke({"input":"Who is playing in the Super Bowl in 2024? Where will it be played?"})



> Entering new AgentExecutor chain...
I should use the Current Event Search tool to find information about the Super Bowl in 2024.
Action: Current Event Search
Action Input: Super Bowl 2024 teams[snippet: The Chiefs and the 49ers will face each other for the second time in five years in Super Bowl 58 at Allegiant Stadium. Find out the date, time, TV channel, odds, spread and history of the teams in the big game., title: Who is in Super Bowl 2024? Here are the teams, odds & spread for Super ..., link: https://www.sportingnews.com/us/nfl/news/super-bowl-2024-teams-odds-spread/6ab6d31aea034ba606eb3c06], [snippet: The Kansas City Chiefs and the San Francisco 49ers will face each other in Super Bowl 2024, a repeat of Super Bowl LIV. Find out how they got there, their previous Super Bowl titles, and how to watch the game live on CBS or Paramount+., title: Who is playing in Super Bowl 2024? - NBC Sports, link: https://www.nbcsports.com/nfl/news/who-is-playing-in-super-bowl-2024], [snippet: 

{'input': 'Who is playing in the Super Bowl in 2024? Where will it be played?',
 'output': 'The Kansas City Chiefs and the San Francisco 49ers will be playing in Super Bowl 2024 at Allegiant Stadium in Las Vegas.'}

In [ ]:
agent_executor.invoke({"input":"Who is playing in the football world cup opening game in 2024? Where will it be played?"})



> Entering new AgentExecutor chain...
I should use the Current Event Search tool to find information about the 2024 football world cup opening game.
Action: Current Event Search
Action Input: 2024 football world cup opening game details[snippet: Discover the 48 teams taking part in the FIFA World Cup 2026 Canada, Mexico, USA™, including host cities, game dates and qualifiers. Learn more with FIFA., title: FIFA World Cup 2026™ | Host Cities, Dates, Teams & more, link: https://www.fifa.com/en/tournaments/mens/worldcup/canadamexicousa2026/articles/fifa-world-cup-2026-hosts-cities-dates-usa-mexico-canada], [snippet: FIFA World Cup 2026 match schedule to be announced on 4 February, title: FIFA World Cup 2026 match schedule to be announced on 4 February, link: https://www.fifa.com/en/tournaments/mens/worldcup/canadamexicousa2026/articles/match-schedule-to-be-announced-4-february-2024], [snippet: Mexico City: Azteca Stadium gets the opening game of the FIFA World Cup, to be played on June 1

{'input': 'Who is playing in the football world cup opening game in 2024? Where will it be played?',
 'output': 'The opening game of the 2024 football world cup will be played at Azteca Stadium in Mexico City on June 11, 2026.'}

In [ ]:
agent_executor.invoke({"input":"who is playing first match of ODI cricket world cup and when will it be played?"})



> Entering new AgentExecutor chain...
I should search for the current event related to the ODI cricket world cup.
Action: Current Event Search
Action Input: ODI cricket world cup first match[snippet: In 1975 the first Cricket World Cup was contested in England as a series of one-day matches of 60 overs per side. It was held outside England, in India and Pakistan, for the first time in 1987. The 1987 contest also saw the number of overs per side reduced to 50. In 2007 Australia became the first team to win three consecutive World Cup ..., title: Cricket World Cup | History, Winners, & Facts | Britannica, link: https://www.britannica.com/sports/Cricket-World-Cup], [snippet: A One Day International (ODI) is a form of 50 overs limited overs cricket, played between two teams with international status, in which each team faces a fixed number of overs, currently 50, with the game lasting up to 7 hours. The Cricket World Cup, generally held every four years, is played in this format.One Day 

{'input': 'who is playing first match of ODI cricket world cup and when will it be played?',
 'output': "The first match of the ODI Cricket World Cup was played on June 7, 1975, between England and India at Lord's Cricket Ground in London."}